<a href="https://colab.research.google.com/github/NikitaSUAI/EmotionRecognition/blob/main/Text_feats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT

In [1]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

# !cp -r '/content/drive/MyDrive/OMG_EMO' 'OMG_EMO'

BASE_PATH = Path("/content/drive/MyDrive/OMG_EMO")

TRAIN_PATH = BASE_PATH / "train_set.csv"
VAL_PATH = BASE_PATH / "val_set.csv"
TEST_PATH = BASE_PATH / "test_set.csv"

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

train_df = pd.read_csv(TRAIN_PATH).dropna()
val_df = pd.read_csv(VAL_PATH).dropna()
test_df = pd.read_csv(TEST_PATH).dropna()

## Word2Vec feats

In [3]:
import gensim.downloader as api

model_wv_gensim = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [36]:
!pip install pymystem3
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))

lemmatizer = WordNetLemmatizer()

def tokenize_lemmatize(sentence):
  sentence = re.sub(r'[^\w\s]',' ',sentence)
  tokens = sentence.lower().strip().split()
  lems = [lemmatizer.lemmatize(token) for token in tokens]
  lems = [token for token in lems if token not in stop_words]
  return lems

def get_w2v_features(text):
  try:
    words = [model_wv_gensim[word] for word in tokenize_lemmatize(text)]
    words = np.stack(words)
    return words
  except :
    return None

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
a = get_w2v_features(train_df.iloc[0].transcript)
a.shape

(7, 300)

In [40]:
import pandas as pd
from tqdm import tqdm
import pickle

tqdm.pandas()

subset = train_df
segment_features = subset.transcript.progress_apply(get_w2v_features)
full_feats = dict()

for f, (_, row) in zip(segment_features, subset.iterrows()):
  full_feats[row.path] = f

with open("/content/drive/MyDrive/OMG_EMO/feats/w2v_train.pkl", "wb") as f:
  pickle.dump(full_feats, f)

100%|██████████| 1660/1660 [00:00<00:00, 9276.71it/s]


In [42]:
subset = val_df
segment_features = subset.transcript.progress_apply(get_w2v_features)
full_feats = dict()

for f, (_, row) in zip(segment_features, subset.iterrows()):
  full_feats[row.path] = f

with open("/content/drive/MyDrive/OMG_EMO/feats/w2v_test.pkl", "wb") as f:
  pickle.dump(full_feats, f)

100%|██████████| 337/337 [00:00<00:00, 1609.81it/s]


## TF-IDF

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = val_df.transcript.to_list() + train_df.transcript.to_list()
corpus = [" ".join(tokenize_lemmatize(text)) for text in corpus]

vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

def get_tf_idf(text):
  text = " ".join(tokenize_lemmatize(text))
  return vectorizer.transform([text]).data


In [52]:
subset = train_df
segment_features = subset.transcript.progress_apply(get_tf_idf)
full_feats = dict()

for f, (_, row) in zip(segment_features, subset.iterrows()):
  full_feats[row.path] = f

with open("/content/drive/MyDrive/OMG_EMO/feats/tf_idf_train.pkl", "wb") as f:
  pickle.dump(full_feats, f)

100%|██████████| 1660/1660 [00:01<00:00, 1392.34it/s]


In [53]:
subset = val_df
segment_features = subset.transcript.progress_apply(get_tf_idf)
full_feats = dict()

for f, (_, row) in zip(segment_features, subset.iterrows()):
  full_feats[row.path] = f

with open("/content/drive/MyDrive/OMG_EMO/feats/tf_idf_test.pkl", "wb") as f:
  pickle.dump(full_feats, f)

100%|██████████| 337/337 [00:00<00:00, 1393.18it/s]
